In [36]:
import os
import matplotlib.pyplot as plt
import pandas as pd
from read_nifti import *
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from model import *

In [59]:
rootdir = r'C://tue//ITP//trial'
csv_file = "C:/tue/ITP/trial.csv"
csv_file = pd.read_csv(csv_file)

# Get folder name at specified index
folder_name = csv_file.iloc[0]['id']
# Get the filename at the specified index
filename_img = csv_file.iloc[0]["p_id"]
filename_seg = csv_file.iloc[0]["p_id_seg"]

# Construct the full path to the image file
image_path = os.path.join(rootdir, folder_name, filename_img)
segmentation_path = os.path.join(rootdir, folder_name, filename_seg)

# MRI scan
image = read_nifti(image_path)

In [39]:
class DatasetMRI(Dataset):
    
    def __init__(self, csv_file, root_dir):
        super(DatasetMRI, self).__init__()
        # train and test csv file
        self.csv_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
    
    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self,index):
        # Get folder name at specified index
        folder_name = self.csv_file.iloc[index]['id']
        # Get the filename at the specified index
        filename_img = self.csv_file.iloc[index]["p_id"]
        filename_seg = self.csv_file.iloc[index]["p_id_seg"]
        
        # Construct the full path to the image file
        image_path = os.path.join(self.root_dir, folder_name, filename_img)
        segmentation_path = os.path.join(self.root_dir, folder_name, filename_seg)
        
        # MRI scan
        image = read_nifti(image_path)
        # Segmentation
        seg = read_nifti(segmentation_path)
        
        #image = np.transpose(image, (0, 2, 1, 3))
        # Get biggest slice in image and segmentation
        biggest_img, biggest_seg = self.get_slice(image, seg)
        
        image_tensor= torch.from_numpy(biggest_img).float()

        # seg = np.transpose(seg, (0, 2, 1, 3))
        seg_tensor = torch.from_numpy(biggest_seg).float()
        
        return image_tensor, seg_tensor
    
    def get_slice(self, image, seg):
        """Function to choose the slice where the biggest tumour map is

        Args:
            image (tensor): MRI image
            seg (tensor): segmentation map
        
        Returns:
            biggest_img (array): corresponds to biggest slice from image
            biggest_seg (array): corresponds to biggest slice from segmentation map
        """
        # image = read_nifti(image)
        # seg = read_nifti(seg)
        
        max_area = 0
        idx_slice = 0

        for i in range(seg.shape[2]):
            slice = seg[:, :, i]
            # Calculate the area of the tumor in the current slice
            tumor_area = np.sum(slice)
            # Update the maximum area and slice index if the current slice has a larger area
            if tumor_area > max_area:
                max_area = tumor_area
                idx_slice = i
        
        # Get biggest slice
        biggest_img = image[:,:,idx_slice]
        biggest_slice = seg[:,:,idx_slice]
        
        return biggest_img, biggest_slice
        
        # return {
        #     'image': torch.as_tensor(image.copy()).float().contiguous(),
        #     'mask': torch.as_tensor(seg.copy()).long().contiguous()
        # }


In [4]:
def get_slice(image, seg):
    """Function to choose the slice where the biggest tumour map is

    Args:
        image (tensor): MRI image
        seg (tensor): segmentation map
    
    Returns:
        biggest_img (array): corresponds to biggest slice from image
        biggest_seg (array): corresponds to biggest slice from segmentation map
    """
    # image = read_nifti(image)
    # seg = read_nifti(seg)
    
    max_area = 0
    idx_slice = 0

    for i in range(seg.shape[2]):
        slice = seg[:, :, i]
        # Calculate the area of the tumor in the current slice
        tumor_area = np.sum(slice)
        # Update the maximum area and slice index if the current slice has a larger area
        if tumor_area > max_area:
            max_area = tumor_area
            idx_slice = i
    
    # Get biggest slice
    biggest_img = image[:,:,idx_slice]
    biggest_slice = seg[:,:,idx_slice]
    
    return biggest_img, biggest_slice
    
    

In [25]:
batch_size = 16

mri_dataset = DatasetMRI(csv_file="C:/tue/ITP/trial.csv",root_dir=rootdir)
train_loader = DataLoader(mri_dataset, batch_size=batch_size, shuffle=True)

In [40]:
for img,lbs in train_loader:
    # Assuming 'images' is a tensor with shape (batch_size, channels, height, width)
    # Convert tensor to numpy array and transpose it to (batch_size, height, width, channels)
    
    images_np = np.transpose(img.numpy(), (0, 2, 1,3))
    label_np = np.transpose(lbs.numpy(), (0, 2, 1,3))
    
    
    # Plot images
    plt.figure()
    plt.imshow(images_np[1,:,:],cmap='gray')
    
    plt.figure()
    plt.imshow(label_np[1,:,:],cmap='gray')

ValueError: axes don't match array

In [58]:
torch.min(image[:,:,54])

tensor(-57.)

In [52]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_input = next(iter(train_loader))[0]  # Get a sample input image from your dataset
#in_channels = sample_input.size(3)  # Update the number of input channels based on the sample input
model = UNet(channels=[1, 64, 128, 256], out_channels=1).to(device)
criterion = nn.CrossEntropyLoss()  # Adjust the loss function as per your task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# For WANDB: https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb#scrollTo=Nizf1NnC7Bbi

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for image, true_mask in train_loader:
        # image, true_mask = batch['image'], batch['mask']
        image = image.unsqueeze(1)  # Add a channel dimension
        true_mask = true_mask.unsqueeze(1)
        # images, true_masks = image.permute(0, 1,), true_mask.permute(0, 2, 3, 1)
        images, masks = image.to(device), true_mask.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")



OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 

In [ ]:
# Validation loop (optional)
model.eval()
with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)

        outputs = model(images)
        # Perform evaluation as required

# Save the trained model
torch.save(model.state_dict(), "unet_model.pth")